In [1]:
import os
from anytree import RenderTree, AsciiStyle, LevelOrderIter

In [10]:
class Node:
    def __init__(self,name,parent=None,children=None,depth=None,ancestors=None):
        self.name = name
        self.parent = parent
        self.children = children
        if depth:
            self.depth = depth
        if ancestors:
            self.ancestors = ancestors
        
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return f"Node({self.name})"
    
    def iterator(self):
        """ iterate tree from self in pre-order depth-first search order """
        yield self
        if self.children:
            for child in self.children:             
                for n in child.iterator():
                    yield n
        else:
            return
        
    def findDepth(self):
        if hasattr(self,"depth"):
            return self.depth
        depth = self.parent.findDepth()+1
        self.depth = depth
        
        return depth
    
    def ancestry(self):
        if hasattr(self,"ancestors"):
            return self.ancestors
        ancestors = self.parent.ancestry()+[self.parent.name]
        self.ancestors = ancestors
        
        return ancestors

In [11]:
def nodeFromName(name,tree_names,tree_nodes):
    for c_name,node in zip(tree_names,tree_nodes):
        if name==c_name:
                return node
    return None

def addChild(parent,child):
    if parent.children:
        parent.children.append(child)
    else:
        parent.children = [child]
    return parent

In [12]:
def createTree(orbits):
    root = Node(orbits[0][0])
    roots = {root}
    for i,[center, satellite] in enumerate(orbits):
        tree_names = [n.name for r in roots for n in r.iterator()]
        tree_nodes = [n for r in roots for n in r.iterator()]
        if (center in tree_names) and (satellite not in tree_names):
            parent = nodeFromName(center,tree_names,tree_nodes)
            sat = Node(satellite,parent=parent)

        elif (center not in tree_names) and (satellite in tree_names):
            parent = Node(center)
            sat = nodeFromName(satellite,tree_names,tree_nodes)
            sat.parent = parent
            roots.add(parent)
        elif (center in tree_names) and (satellite in tree_names):
            parent = nodeFromName(center,tree_names,tree_nodes)
            sat = nodeFromName(satellite,tree_names,tree_nodes)
            sat.parent = parent
        else:
            parent = Node(center)
            sat = Node(satellite,parent=parent)
            roots.add(parent)
        addChild(parent,sat)

        while root.parent:
            root = root.parent

        roots.add(root)
        if sat in roots:
            roots.remove(sat)
            
    root.depth=0
    root.ancestors=[]
    return root

In [13]:
filename = os.path.join(os.getcwd(),"d6_input")
with open(filename) as f:
    orbits=[orbit.strip().split(")") for orbit in f.readlines()]

In [14]:
import random
str_input = "COM)B,B)C,C)D,D)E,E)F,B)G,G)H,D)I,E)J,J)K,K)L,K)YOU,I)SAN".split(",")
random.shuffle(str_input)
orbits = [orbit.split(")") for orbit in str_input]
print(orbits)

[['I', 'SAN'], ['B', 'G'], ['COM', 'B'], ['G', 'H'], ['E', 'F'], ['B', 'C'], ['D', 'E'], ['J', 'K'], ['D', 'I'], ['K', 'YOU'], ['E', 'J'], ['C', 'D'], ['K', 'L']]


In [15]:
root = createTree(orbits)
# print(RenderTree(root))

In [16]:
def orbitsCount(root):
    orbits_sum = 0
    for n in root.iterator():
        orbits_sum+=n.findDepth()
    return orbits_sum

orbitsCount(root)

54

In [27]:
anc_list =[]
for n in root.iterator():
    if n.name=="SAN" or n.name=="YOU":
        anc_list.append(n.ancestry())

i=0
while anc_list[0][i]== anc_list[1][i]:
    i+=1
print(anc_list)
print(i)
len(anc_list[0])+len(anc_list[1][i])-2*i

[['COM', 'B', 'C', 'D', 'E', 'J', 'K'], ['COM', 'B', 'C', 'D', 'I']]
4


0

In [26]:
len(anc_list[1])-i

1